<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/HW_05_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 05 NLP

In [ ]:
import numpy as np

Берем отызывы за лето (из архива с материалами или предыдущего занятия)

## Задание 1

1. Учим conv сеть для классификации - выбить auc выше 0.95

## Задание 2

2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?